Данный ноутбук исполнялся в среде разработки Google Colab Pro с 52 Гб ОЗУ, CPU only

Присоединяем Google Диск

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Импорт библиотек

In [ ]:
import os
import pickle
import joblib
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score



RANDOM_SEED = 42

HACKATHON_CASE_PATH = "/content/drive/MyDrive/Кейс-2. Склонность физических лиц к инвестициям/"
HACKATHON_META_PATH = "/content/drive/MyDrive/hacks/alpha_hack_2024"

np.random.seed(RANDOM_SEED)

## Чтение обучающих данных

Считываем предикты наших лучших моделей (Catboost + XGBoost + LightGBM)

In [ ]:
for_stacking_path = os.path.join(HACKATHON_META_PATH, "for_stacking")
os.listdir(for_stacking_path)

['lgbm_85.47_test_preds.csv',
 'lgbm_85.47_train_preds.csv',
 'catboost_85.85_train_preds.csv',
 'catboost_85.85_test_preds.csv',
 'catboost_86.03_test_preds.csv',
 'train_targets.csv',
 'catboost_86.03_train_preds.csv',
 'xgb_85.94_train_preds.csv',
 'xgb_85.94_test_preds.csv',
 'xgb_85.99_train_preds.csv',
 'xgb_85.99_test_preds.csv',
 'catboost_85.99_test_preds.csv',
 'catboost_85.99_train_preds.csv',
 'catboost_86.11_test_preds.csv',
 'catboost_86.11_train_preds.csv',
 'lgbm_85.78_test_preds.csv',
 'lgbm_85.78_train_preds.csv',
 'lgbm_85.9_test_preds.csv',
 'lgbm_85.9_train_preds.csv']

In [ ]:
train_catboost_86_11_preds = pd.read_csv(os.path.join(for_stacking_path, 'catboost_86.11_train_preds.csv'))
train_catboost_86_11_preds.head()

,id,catboost_86.11_proba
0,0,0.011543
1,1,0.002898
2,2,0.003532
3,3,0.015211
4,4,0.001515


In [ ]:
train_xgb_85_99_preds = pd.read_csv(os.path.join(for_stacking_path, 'xgb_85.99_train_preds.csv'))
train_xgb_85_99_preds = train_xgb_85_99_preds.rename(columns={'xgboost_proba': 'xgboost_85.99_proba'})
train_xgb_85_99_preds.head()

,id,xgboost_85.99_proba
0,9,0.227028
1,15,0.021789
2,31,0.146717
3,32,0.013526
4,35,0.116487


In [ ]:
train_lgbm_85_9_preds = pd.read_csv(os.path.join(for_stacking_path, 'lgbm_85.9_train_preds.csv'))
train_lgbm_85_9_preds = train_lgbm_85_9_preds.rename(columns={'lgbm_proba': 'lgbm_85.9_proba'})
train_lgbm_85_9_preds.head()

,id,lgbm_85.9_proba
0,0,0.014030
1,1,0.002915
2,2,0.003472
3,3,0.014260
4,4,0.002229


Считываем таргеты для джоина и использования в обучении

In [ ]:
train_targets = pd.read_csv(os.path.join(for_stacking_path, 'train_targets.csv'))
train_targets.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


Джойним таргеты с предсказаниями лучших моделей на размеченной обучающей выборке

In [ ]:
df = train_targets.merge(train_catboost_86_11_preds, on='id') \
                  .merge(train_xgb_85_99_preds, on='id') \
                  .merge(train_lgbm_85_9_preds, on='id') \
                  .drop('id', axis=1)

df

,target,catboost_86.11_proba,xgboost_85.99_proba,lgbm_85.9_proba
0,0,0.011543,0.191637,0.014030
1,0,0.002898,0.032831,0.002915
2,0,0.003532,0.074489,0.003472
3,0,0.015211,0.185737,0.014260
4,0,0.001515,0.023157,0.002229
...,...,...,...,...
4490463,1,0.996894,0.998519,0.992674
4490464,0,0.070457,0.609010,0.058224
4490465,0,0.213026,0.819924,0.269011
4490466,0,0.019184,0.269562,0.025646


Пробовали составлять различные фичи, но они особо не дали прироста на публичном лидерборде

In [ ]:
# df['catboost_xgb_mul'] = df['catboost_86.03_proba'] * df['xgboost_85.99_proba']
# df['catboost_lgbm_mul'] = df['catboost_86.03_proba'] * df['lgbm_85.47_proba']
# df['xgb_lgbm_mul'] = df['xgboost_85.99_proba'] * df['lgbm_85.47_proba']

# df['catboost_xgb_div'] = df['catboost_86.03_proba'] / df['xgboost_85.99_proba']
# df['catboost_lgbm_div'] = df['catboost_86.03_proba'] / df['lgbm_85.47_proba']
# df['xgb_lgbm_div'] = df['xgboost_85.99_proba'] / df['lgbm_85.47_proba']

# df['catboost_xgb_sum'] = df['catboost_86.03_proba'] + df['xgboost_85.99_proba']
# df['catboost_lgbm_sum'] = df['catboost_86.03_proba'] + df['lgbm_85.47_proba']
# df['xgb_lgbm_sum'] = df['xgboost_85.99_proba'] + df['lgbm_85.47_proba']

df.head()

,target,catboost_86.11_proba,xgboost_85.99_proba,lgbm_85.9_proba
0,0,0.011543,0.191637,0.014030
1,0,0.002898,0.032831,0.002915
2,0,0.003532,0.074489,0.003472
3,0,0.015211,0.185737,0.014260
4,0,0.001515,0.023157,0.002229


## Подготовка данных

Делаем сплит данных в соотношении 9:1, по нашему сиду, с сохранением доли единиц в таргете

In [ ]:
from sklearn.model_selection import train_test_split


X, y = df.drop('target', axis=1), df['target']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((4041421, 3), (449047, 3), (4041421,), (449047,))

Нормализуем данные (вычисляем среднее и стандартное отклонение на обучающей выборке)

In [ ]:
X_mean = np.mean(X_train.values, axis=0)
X_std = np.std(X_train.values, axis=0) + 1e-8

X_train_values = (X_train.values - X_mean) / X_std
X_valid_values = (X_valid.values - X_mean) / X_std

Обучаем SVM, гиперпараметры были подобраны вручную с целью максимизации ROC-AUC на валидации.

Для стекинга также были опробованы Логистическая регрессия и кастомная нейронная сеть с 1-2 слоями с сигмоидой на конце, но они показали хуже результаты, как на валидации так и на публичном лидерборде.

In [ ]:
from sklearn.svm import LinearSVC


clf = LinearSVC(
    tol=1e-7, C=1e-5, random_state=42, max_iter=100000,
    # class_weight='balanced', verbose=True
)
clf.fit(X_train_values, y_train)

LinearSVC(C=1e-05, max_iter=100000, random_state=42, tol=1e-07)

Один из лучших показателей на валидации

In [ ]:
from sklearn.metrics import roc_auc_score


y_pred = clf.decision_function(X_valid_values)
roc_auc_score(y_valid, y_pred)

0.8649074674097154

Интересно посмотреть как SVM делает предикты

In [ ]:
y_pred

array([-0.93165108, -0.96844182, -0.96567952, ..., -0.98117181,
       -0.96940564, -0.93280802])

Переобучаем модель на всей выборке, на этот раз среднее и стандартное отклонение для нормализации также высчитываем по всей выборке. Переобучаем чтобы модель увидела все примеры и скор на тесте был еще выше

In [ ]:
# Обучаем на всей выборке
X_mean = np.mean(X.values, axis=0)
X_std = np.std(X.values, axis=0) + 1e-8

X_scaled = X.values
X_scaled = (X_scaled - X_mean) / X_std

clf = LinearSVC(tol=1e-7, C=1e-5, random_state=42, max_iter=100000, verbose=True)
clf.fit(X_scaled, y)

[LibLinear]

LinearSVC(C=1e-05, max_iter=100000, random_state=42, tol=1e-07, verbose=True)

## Считываем тестовые данные

In [ ]:
test_catboost_86_11_preds = pd.read_csv(os.path.join(for_stacking_path, 'catboost_86.11_test_preds.csv'))
test_catboost_86_11_preds.head()

,id,target
0,4490468,0.010694
1,4490469,0.008120
2,4490470,0.013523
3,4490471,0.117551
4,4490472,0.407733


In [ ]:
test_xgb_85_99_preds = pd.read_csv(os.path.join(for_stacking_path, 'xgb_85.99_test_preds.csv'))
test_xgb_85_99_preds.head()

,id,target
0,4490468,0.121333
1,4490469,0.123818
2,4490470,0.106819
3,4490471,0.518116
4,4490472,0.938831


In [ ]:
test_lgbm_85_9_preds = pd.read_csv(os.path.join(for_stacking_path, 'lgbm_85.9_test_preds.csv'))
test_lgbm_85_9_preds

,id,target
0,4490468,0.010243
1,4490469,0.012519
2,4490470,0.009356
3,4490471,0.077187
4,4490472,0.573810
...,...,...
509527,4999995,0.029924
509528,4999996,0.007762
509529,4999997,0.026947
509530,4999998,0.064388


In [ ]:
test_catboost_86_11_preds = test_catboost_86_11_preds.rename(columns={'target': 'catboost_86.11_proba'})
test_xgb_85_99_preds = test_xgb_85_99_preds.rename(columns={'target': 'xgboost_85.99_proba'})
test_lgbm_85_9_preds = test_lgbm_85_9_preds.rename(columns={'target': 'lgbm_85.9_proba'})

Джойним все скоры

In [ ]:
test = test_catboost_86_11_preds \
                .merge(test_xgb_85_99_preds, on='id') \
                .merge(test_lgbm_85_9_preds, on='id')

test

,id,catboost_86.11_proba,xgboost_85.99_proba,lgbm_85.9_proba
0,4490468,0.010694,0.121333,0.010243
1,4490469,0.008120,0.123818,0.012519
2,4490470,0.013523,0.106819,0.009356
3,4490471,0.117551,0.518116,0.077187
4,4490472,0.407733,0.938831,0.573810
...,...,...,...,...
509527,4999995,0.080105,0.426787,0.029924
509528,4999996,0.010839,0.095272,0.007762
509529,4999997,0.025132,0.329761,0.026947
509530,4999998,0.035854,0.433350,0.064388


Проверяем что все ОК

In [ ]:
# test['catboost_xgb_mul'] = test['catboost_86.03_proba'] * test['xgboost_85.99_proba']
# test['catboost_lgbm_mul'] = test['catboost_86.03_proba'] * test['lgbm_85.47_proba']
# test['xgb_lgbm_mul'] = test['xgboost_85.99_proba'] * test['lgbm_85.47_proba']

# test['catboost_xgb_div'] = test['catboost_86.03_proba'] / test['xgboost_85.99_proba']
# test['catboost_lgbm_div'] = test['catboost_86.03_proba'] / test['lgbm_85.47_proba']
# test['xgb_lgbm_div'] = test['xgboost_85.99_proba'] / test['lgbm_85.47_proba']

# df['catboost_xgb_sum'] = df['catboost_86.03_proba'] + df['xgboost_85.99_proba']
# df['catboost_lgbm_sum'] = df['catboost_86.03_proba'] + df['lgbm_85.47_proba']
# df['xgb_lgbm_sum'] = df['xgboost_85.99_proba'] + df['lgbm_85.47_proba']

test.head()

,id,catboost_86.11_proba,xgboost_85.99_proba,lgbm_85.9_proba
0,4490468,0.010694,0.121333,0.010243
1,4490469,0.008120,0.123818,0.012519
2,4490470,0.013523,0.106819,0.009356
3,4490471,0.117551,0.518116,0.077187
4,4490472,0.407733,0.938831,0.573810


Создаем сабмит отдельным DataFrame'ом и дропаем id

In [ ]:
submit = test[['id']]
test = test.drop('id', axis=1)

Нормализуем тестовые данные

In [ ]:
X_test_values = (test.values - X_mean) / X_std

Делаем предсказание нашей моделью SVM

In [ ]:
submit['target'] = clf.decision_function(X_test_values)

<ipython-input-34-4b66d4ec4177>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['target'] = clf.decision_function(X_test_values)


In [ ]:
submit

,id,target
0,4490468,-0.970603
1,4490469,-0.973861
2,4490470,-0.966920
3,4490471,-0.750299
4,4490472,-0.059180
...,...,...
509527,4999995,-0.834609
509528,4999996,-0.972491
509529,4999997,-0.930946
509530,4999998,-0.894064


Сохраняем предсказание на диске

In [ ]:
submissions_path = os.path.join(HACKATHON_META_PATH, "submissions")
os.makedirs(submissions_path, exist_ok=True)

submit.to_csv(os.path.join(submissions_path, "stacking_svm_best_models_12112024.csv"), index=False)